In [103]:
from haystack import Document, Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore

from haystack_integrations.components.generators.ollama import OllamaGenerator

In [106]:
arr = []
for x in range(1, 4):
    with open("/Users/gaurishlakhanpal/Downloads/HW" + str(x) + "_README.md", "r") as f:
        content = f.read()
    arr.append(Document(content = content, meta = {'title': "Homework " + str(x), 'type': "hw"}))
    f.close()

In [107]:
arr[2]

Document(id=4774bffd4da0c1d090d299b9fe22ba45dd9857d66e4e0bf1d36696837898abfc, content: '# Homework 3: Histograms and Distributions

This homework asks you to write and use the functions we...', meta: {'title': 'Homework 3', 'type': 'hw'})

In [108]:
arr.append(Document(content="Note: <p>Hi all!</p><p></p><p>All projects submitted to Gradescope have now been graded, feedback published, and grades posted to Brightspace. If your grade on Brightspace for the project is a zero it may be for the following reasons:</p><ol><li>You, or your partner, did not fill out the survey in the project Readme indicating the members of your team</li><li>You, or your partner, filled out the survey incorrectly</li><li>You, or your partner, did not submit your project writeup to Gradescope and did not contact an instructor to get it uploaded during the grading period</li></ol><p>IMPORTANT: We are allowing for a 24-hour period in which students can make regrade requests regarding the project. consequently, we will not accept any regrade request made after 11:59 PM tomorrow (12/18). Please check your grade on Brightspace and make the appropriate requests to the course email IMMEDIATELY. </p><p></p><p>Have a great winter break everyone!</p><p></p><p>Best,</p><p></p><p>Brooks</p>", 
                    meta={'folders': ['project'], 'title': 'Project Grades Released! [IMPORTANT, 24-HOUR REGRADE REQUEST PERIOD]', 'type': "hw"}))
arr.append(Document(content="Note: <p>Hi all!</p><p></p><p>As you are preparing your project write-up for submission, please take some time to evaluate your write-up/code using the general rubric posted below. We will be instructing the UTAs to evaluate your write-ups according to these five categories and a write-up that is acceptable for all criteria will be given full points. Good luck with finals and have a great rest of the semester!</p><p></p><p>Best,</p><p></p><p>Brooks</p><p></p><p><a href='/redirect/s3?bucket=uploads&amp;prefix=paste%2Fktit8n4gsw8346%2F14403a48ac263ab5666762e37ea9a6fa9010a0b04e0d9704d30c56e13230701e%2FECE2075_ProjectRubric.pdf' target='_blank' rel='noopener noreferrer'>ECE2075_ProjectRubric.pdf</a></p>", meta={'folders': ['project'], 'title': 'General Rubric for Project Write-up', 'type': "hw"}))
arr.append(Document(content="Note: <p>Dear All,</p><p><br />Some of you have inquired about the cutoffs between different letter grades. While our policy is not to release the exact cutoffs, we will make the following worst-case guarantees on the mapping from your final number grade (after weighting the assignments, exams, and project) to letter grade:</p><p style='padding-left:40px'>A-range: &gt;90<br />B-range: &gt;80<br />C-range: &gt;70<br />D-range: &gt;60</p><p>Note that these are worst case, i.e., your actual letter grade will be greater than or equal to that given by these ranges. Different cutoffs might be used to benefit you depending on the grade distribution and other various factors.</p><p><br />Hope it helps!</p>", meta={'folders': ['logistics'], 'title': 'Regarding Final Grade Cutoffs', 'type': "hw"}))
arr.append(Document(content="Note: <script type='text/javascript'>PA.load('/dashboard/project_partners', null, function(data){ $('#' + 'questionText').html(data);});</script> #pin", meta={'folders': [], 'title': 'Search for Teammates!', 'type': "hw"}))

In [109]:
document_store = InMemoryDocumentStore()
document_store.write_documents(arr)

7

In [110]:
template = """
Given only the following information, answer the question.
Ignore your own knowledge.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ query }}?
"""

In [111]:
pipe = Pipeline()

pipe.add_component("retriever", InMemoryBM25Retriever(document_store=document_store))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", OllamaGenerator(model="llama3.1", url="http://localhost:11434"))
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")

query = "What are the tasks for homework 1?"

response = pipe.run({"prompt_builder": {"query": query}, "retriever": {"query": query}})

print(response["llm"]["replies"][0])

## Homework 1 Tasks

You have completed all the tasks for homework 1. The specific tasks were:

*   Implementing a `histogram` function that takes in a list of numbers and returns a dictionary where each key is a number in the list and its corresponding value is how many times it appears.
*   Creating an `addressbook` function that takes in two lists, one containing people's names and another containing their phone numbers. The function should return a dictionary with addresses as keys and values being tuples of (list of names, phone number). If there are multiple names associated with the same address but different numbers, the function uses the first name encountered in its output.

You have successfully completed these tasks!


In [7]:
import os
import time
from piazza_api import Piazza

def scrape_course(course_id, course_name):
    p = Piazza()

    email = input("Enter Piazza email: ")
    password = input("Enter Piazza password: ")
    p.user_login(email=email, password=password)

    course = p.network(course_id)

    with open(str(course_name)+"_Piazza_Data.txt", "w") as file:
        file.write("document_store = InMemoryDocumentStore()\ndocument_store.write_documents(\n\t[\n")

    cnt = 0

    for post in course.iter_all_posts():
        folders = post['folders']
        timestamp = post['created']
        title = post['history'][0]['subject']
        unformatted_content = post['history'][0]['content']
        content = unformatted_content.replace("\n", "").replace("\"", "'")
        tags = post['tags']
        likes = len(post['tag_good'])
        views = post['unique_views']

        is_anonymous = post['history'][0]['anon']
        if is_anonymous == "no":
            user = post['history'][0]['uid']
        else:
            user = ""

        metadata = {
            "folders": folders,
            "title": title,
            "timestamp": timestamp,
            "user": user,
            "tags": tags,
            "likes": likes,
            "views": views
        }
        
        is_note = post['type']
        if is_note == "question":
            reply = ""
            for r in post['children']:
                if (r['type'] == 'i_answer') or (r['type'] == 's_answer' and not reply):
                    unformatted_reply = r['history'][0]['content']
                    reply = unformatted_reply.replace("\n", "").replace("\"", "'")
            if reply:
                document_representation = f"""\t\tDocument(content="Question: {content} Answer: {reply}", meta={metadata}),\n"""
            else:
                continue    # do not use unanswered questions
        else:
            document_representation = f"""\t\tDocument(content="Note: {content}", meta={metadata}),\n"""

        with open(str(course_name)+"_Piazza_Data.txt", "a") as file:
            file.write(document_representation)

        cnt += 1

        if cnt == 50:   # rate limit seems to be near 60/min
            time.sleep(60)
            cnt = 0


if __name__ == "__main__":
    course_id = "l70jo8vm9bk6mu"
    course_name = "20875_F22"   # just for text file name
    scrape_course(course_id, course_name)

Enter Piazza email:  glakhanp@purdue.edu
Enter Piazza password:  JeromePowell123$


RequestError: Could not retrieve your feed.
Response: {
  "result": null,
  "error_codes": [],
  "error": "No permission",
  "aid": "m22f3qqph9812c"
}